In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time
import json


In [3]:
driver = webdriver.Chrome()
driver.get("https://masscannabiscontrol.com/calendar/")
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [5]:
yearly_button = driver.find_element(By.CSS_SELECTOR, "span.mec-totalcal-yearlyview")
yearly_button.click()

In [6]:
# load button
def load_all_meetings():
    count = 0
    while True:
        try:
            load_more = driver.find_element(By.CSS_SELECTOR, "div.mec-load-more-button")
            load_more.click()
            time.sleep(2)
            count += 1
            if count > 5: # 5 is worst case scenario
                break
        except:
            break
    

In [21]:
urls = []

In [16]:
def select_option(select_id, value):
    # Locate the <select> tag by its ID
    select_element = driver.find_element(By.ID,select_id)

    # Initialize the Select object
    select = Select(select_element)

    # Select the option by visible text
    select.select_by_visible_text(str(value))

In [22]:
for year in years:
    select_option("mec_sf_year_348", year)
    time.sleep(2)
    select_option("mec_sf_month_348", "January") # January sets the stage for the upcoming months
    time.sleep(2)

    load_all_meetings()
    meetings = driver.find_elements(By.CSS_SELECTOR, "div.mec-yearly-agenda-sec a")

    for meeting in meetings:
        urls.append(meeting.get_attribute("href"))

In [23]:
print(f"Found {len(urls)} urls")

Found 336 urls


In [24]:
print(urls)

['https://masscannabiscontrol.com/events/cannabis-control-commission-public-meeting-2/?occurrence=2017-09-12', 'https://masscannabiscontrol.com/events/cannabis-control-commission-public-meeting-9/?occurrence=2017-09-26', 'https://masscannabiscontrol.com/events/invited-stakeholder-listening-session/?occurrence=2017-10-02', 'https://masscannabiscontrol.com/events/cannabis-control-commission-public-meeting-10/?occurrence=2017-10-03', 'https://masscannabiscontrol.com/events/public-listening-session/?occurrence=2017-10-04', 'https://masscannabiscontrol.com/events/public-listening-session-2/?occurrence=2017-10-05', 'https://masscannabiscontrol.com/events/public-listening-session-3/?occurrence=2017-10-06', 'https://masscannabiscontrol.com/events/cannabis-control-commission-executive-director-search-subcommittee-public-meeting/?occurrence=2017-10-06', 'https://masscannabiscontrol.com/events/4255-2/?occurrence=2017-10-10', 'https://masscannabiscontrol.com/events/public-listening-session-4/?occu

In [20]:
# Save URLs to a JSON file
with open('urls.json', 'w') as file:
    json.dump(urls, file, indent=4)

In [25]:
driver.quit()

In [107]:
# Load URLs from the JSON file
with open('urls.json', 'r') as file:
    urls = json.load(file)


In [108]:
def get_page_data(url, driver):
    driver.get(url)
    # Wait to load page
    time.sleep(2)

    try:
    
        header_title = driver.find_element(By.CSS_SELECTOR, 'h1.mec-single-title').text

        related_documents = driver.find_element(By.CSS_SELECTOR, 'div.av_one_third')

        event_info = driver.find_element(By.CSS_SELECTOR, 'div.mec-event-meta')
        date = event_info.find_element(By.CSS_SELECTOR, 'div.mec-single-event-date>dd').text
        event_time = event_info.find_element(By.CSS_SELECTOR, 'div.mec-single-event-time>dd').text
        labels = event_info.find_elements(By.CSS_SELECTOR, 'div.mec-single-event-label>dd')
        labels = [label.text for label in labels]

        contents = driver.find_element(By.CSS_SELECTOR, 'div.av_two_third')
        

    except Exception as e:
        return None


    content_dict = {}

    content_dict['title'] = header_title
    content_dict['url'] = url

    content_dict['date'] = date
    content_dict['time'] = event_time
    content_dict['labels'] = labels


    try:
        
        element = contents.find_element(By.XPATH, "//p[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'location')]")
        location = element.text
        while element and element.tag_name == 'p':
            element = element.find_element(By.XPATH, "following-sibling::*[1]")
            location += " " + element.text

        content_dict['location'] = ":".join(location.split(':')[1:]).strip()
    except:
        print("No location found")


    related_documents_links = related_documents.find_elements(By.TAG_NAME, 'a')
    related_documents_dict = []
    for link in related_documents_links:
        related_documents_dict.append({
            'title': link.text,
            'url': link.get_attribute('href')
        })

    content_dict['related_documents'] = related_documents_dict

    return content_dict



    

In [109]:
# Initialize the driver
driver = webdriver.Chrome()

In [110]:
results = []

In [111]:
print(f'number of urls: {len(urls)}')

for i in range(0, len(urls)):
    print(f"Processing {i}/{len(urls)}")
    data = get_page_data(urls[i], driver)
    results.append(data)

number of urls: 336
Processing 0/336
Processing 1/336
Processing 2/336
Processing 3/336
Processing 4/336
Processing 5/336
Processing 6/336
Processing 7/336
Processing 8/336
Processing 9/336
Processing 10/336
Processing 11/336
Processing 12/336
Processing 13/336
Processing 14/336
Processing 15/336
Processing 16/336
Processing 17/336
Processing 18/336
Processing 19/336
Processing 20/336
Processing 21/336
Processing 22/336
Processing 23/336
Processing 24/336
Processing 25/336
Processing 26/336
Processing 27/336
Processing 28/336
Processing 29/336
Processing 30/336
Processing 31/336
No location found
Processing 32/336
No location found
Processing 33/336
Processing 34/336
Processing 35/336
Processing 36/336
Processing 37/336
Processing 38/336
Processing 39/336
Processing 40/336
Processing 41/336
Processing 42/336
Processing 43/336
Processing 44/336
Processing 45/336
Processing 46/336
Processing 47/336
Processing 48/336
Processing 49/336
Processing 50/336
Processing 51/336
Processing 52/336


In [112]:
results = [item for item in results if item is not None]

In [117]:
len([item for item in results if 'location' not in item])

75

In [114]:
# Save the results to a JSON file
with open('events.json', 'w') as file:
    json.dump(results, file, indent=4)

In [116]:
# Cleanup
driver.quit()